## Mandate-4 Contributions
### Notebook-2

In this notebook, we will train the model on random PAWS data in which I have shuffled the columns and rows. Downloading PAWS data:--

In [ ]:
!mkdir data
!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
!tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
!mv data/final/* data
!rm -r data/final

--2023-05-04 02:39:08--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.194.128, 173.194.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz’

paws_wiki_labeled_f 100%[===================>]   4.47M  --.-KB/s    in 0.04s   

2023-05-04 02:39:08 (110 MB/s) - ‘data/paws_wiki_labeled_final.tar.gz’ saved [4687157/4687157]

final/test.tsv
final/
final/train.tsv
final/dev.tsv
--2023-05-04 02:39:09--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Len

Here we will install evaluate and rouge-score libraries for checking accuracy of our model.

In [ ]:
!pip install simpletransformers
!pip install utils
!pip install transformers==4.23.1

!pip install evaluate
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21

In [ ]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


Functions for loading the data and pre-processing the datasets.

In [ ]:
import warnings

import pandas as pd


def load_data(file_path, input_text_column, target_text_column, label_column, keep_label=1):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
        .replace("@", "")
        .replace("#", "")
        .replace("*", "")
    )
    return out_string


Loading Google PAWS data

In [ ]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"


In [ ]:
train_df

,input_text,target_text,prefix
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,paraphrase
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,paraphrase
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,paraphrase
5,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",paraphrase
7,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,paraphrase
...,...,...,...
49384,"The Romanesque language , Galician ( Galego ) ...",The Romance language currently spoken in Galic...,paraphrase
49390,Note that k is a vector consisting of three in...,It is necessary to note that k is a vector con...,paraphrase
49393,"Tim Henman won in the final 6 -- 2 , 7 -- 6 , ...","Tim Tim Henman won 6 -- 2 , 7 -- 6 against Yev...",paraphrase
49395,He was considered an active member of the coun...,He was considered an active member of the Coun...,paraphrase


In [ ]:

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

train_df

           prefix                                        target_text  \
1      paraphrase  The NBA season of 1975 -- 76 was the 30th seas...   
3      paraphrase  When comparable rates of flow can be maintaine...   
4      paraphrase  It is the seat of Zerendi District in Akmola R...   
5      paraphrase  William Henry Henry Harman was born on 17 Febr...   
7      paraphrase  With a discrete amount of probabilities Formul...   
...           ...                                                ...   
49384  paraphrase  The Romanesque language, Galician ( Galego ), ...   
49390  paraphrase  Note that k is a vector consisting of three in...   
49393  paraphrase  Tim Henman won in the final 6 -- 2, 7 -- 6, ag...   
49395  paraphrase  He was considered an active member of the coun...   
49397  paraphrase  She was in Cork on June 24 and arrived on 8 Ju...   

                                              input_text  
1      The 1975 -- 76 season of the National Basketba...  
3      The result

,prefix,target_text,input_text
1,paraphrase,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
3,paraphrase,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
4,paraphrase,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
5,paraphrase,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro, V..."
7,paraphrase,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...
...,...,...,...
49384,paraphrase,"The Romanesque language, Galician ( Galego ), ...",The Romance language currently spoken in Galic...
49390,paraphrase,Note that k is a vector consisting of three in...,It is necessary to note that k is a vector con...
49393,paraphrase,"Tim Henman won in the final 6 -- 2, 7 -- 6, ag...","Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg..."
49395,paraphrase,He was considered an active member of the coun...,He was considered an active member of the Coun...


Shuffling the columns and rows. We will take 2000 rows of data.

In [ ]:
train_df.rename(columns = {'input_text':'target_text1'}, inplace = True)
train_df.rename(columns = {'target_text':'input_text'}, inplace = True)
train_df.rename(columns = {'target_text1':'target_text'}, inplace = True)

In [ ]:
train_df

,prefix,target_text,input_text
1,paraphrase,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
3,paraphrase,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
4,paraphrase,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
5,paraphrase,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro, V..."
7,paraphrase,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...
...,...,...,...
49384,paraphrase,"The Romanesque language, Galician ( Galego ), ...",The Romance language currently spoken in Galic...
49390,paraphrase,Note that k is a vector consisting of three in...,It is necessary to note that k is a vector con...
49393,paraphrase,"Tim Henman won in the final 6 -- 2, 7 -- 6, ag...","Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg..."
49395,paraphrase,He was considered an active member of the coun...,He was considered an active member of the Coun...


In [ ]:
print(type(train_df))
train_df_shuffled = train_df.sample(frac = 1)
train_df_shuffled = train_df_shuffled.head(2000)
eval_df = eval_df.head(250)

In [ ]:
eval_df

,prefix,input_text,target_text
1,paraphrase,They were there to enjoy us and they were ther...,They were there for us to enjoy and they were ...
2,paraphrase,"After the end of the war in June 1902, Higgins...","In August, after the end of the war in June 19..."
3,paraphrase,From the merger of the Four Rivers Council and...,Shawnee Trails Council was formed from the mer...
4,paraphrase,The group toured extensively and became famous...,The group toured extensively and was famous in...
5,paraphrase,Kathy and her husband Pete Beale ( Peter Dean ...,Kathy and her husband Peter Dean ( Pete Beale ...
...,...,...,...
548,paraphrase,"Like the Pacific Maritime Ecozone, Coquitlam l...","Like Pacific Maritime Ecozone, Coquitlam is in..."
549,paraphrase,"The nations of Macedonia, Kenya, Azerbaijan, U...","The nations Macedonia, Kenya, Azerbaijan, Urug..."
552,paraphrase,"On 29 November 1171, Gonzalo signed a document...","On 29 November 1171, Gonzalo signed a charter ..."
554,paraphrase,The remains of two Armenian churches still pre...,The remains of two still preserved Armenian ch...


In [ ]:
train_df_shuffled

,prefix,target_text,input_text
35288,paraphrase,He graduated from the Military School in Sofia...,He graduated from the Military School in Sofia...
31457,paraphrase,"He developed players like Stefan Bogomilov , B...","He developed players like Stefan Bogomilov , B..."
40845,paraphrase,"When the police made the discovery , Latimer d...","When the police made the discovery , Latimer d..."
18572,paraphrase,The school is headed by the Neelaveni Thayaram...,The school is managed by the Neelaveni Thayara...
28385,paraphrase,Later he joined Calcutta Football League and p...,He later joined the Calcutta Football League a...
...,...,...,...
9882,paraphrase,Flashpoint : Cold War Crisis is a tactical sho...,Operation Flashpoint : Cold War Crisis is a ta...
37936,paraphrase,kinetic compensation : an increase in the pree...,Kinetic Compensation : An increase in the pree...
45912,paraphrase,The defect in the mechanism could not be detec...,The defect of the mechanism could not be seen ...
2511,paraphrase,"The participants included TJ Trinidad , Joem B...","TJ Trinidad , Biboy Ramirez , Eric Fructuoso ,..."


In [ ]:

model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 4
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.do_sample = True
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Here, we are again training the model which was previously trained on MSR data in previous notebook.

In [ ]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="/content/gdrive/My Drive/2MayNewModel/best_model",
    args=model_args
)

model.train_model(train_df_shuffled, eval_data=eval_df)

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

...

Running Epoch 2 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

(1000,
 {'global_step': [250, 500, 750, 1000],
  'eval_loss': [0.17218295112252235,
   0.16605503857135773,
   0.16605303436517715,
   0.16605303436517715],
  'train_loss': [0.14816062152385712,
   0.106998011469841,
   0.09026952087879181,
   0.10656018555164337]})

Saving the model in different folder in google drive :--

In [ ]:
# !pwd
# !cd /content/outputs && ls -l
# !ls -l
!cp -r "/content/outputs/best_model" "/content/gdrive/My Drive/4May"

Testing the new Model :--

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("/content/gdrive/My Drive/4May/best_model", forced_bos_token_id=0)
tok = BartTokenizer.from_pretrained("/content/gdrive/My Drive/4May/best_model")

example_english_phrase = "Transformers Transformers are fast plus efficient"
batch = tok(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])
generated_sentence = tok.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_sentence)

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(encoder_decoder_type="bart",
                     encoder_decoder_name="/content/gdrive/My Drive/4May/best_model"
                     ,use_cuda=False
                     )

to_predict = ["They were there to enjoy us and they were there to pray for us.",
              "What is the best way to play Cricket?",
              "This is the best way to create food.",
              "What are different ways to play badminton?",
              "We should not talk in rude manner to guests.",
              "I do not know how many times I have prepared this model.",
              "This is how you make cake",
              "We remove special symbols like #, @, *, etc. We cannot remove useful punctuations like question mark because the paraphrase of a question will be a question only. We use  regex or python string functions.",
              "Amrozi accused his brother, whom he called \"the witness\", of deliberately distorting  his evidence.",
              "What are the ways to commit suicide?",
              "How can I get started using Quora?",
              "How to play chess?"
              ]

print(type(to_predict))
predicted = []
j = 0
# while True:
for i in to_predict:
    to_predict1 = [i]

    preds = model.predict(to_predict1)

    predicted.insert(j, preds[0])
    print("---------------------------------------------------------")
    print(i)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)
        # predicted[j].append(pred)
    
    j = j+1
    print("---------------------------------------------------------")
    print()

<class 'list'>


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
They were there to enjoy us and they were there to pray for us.

Predictions >>>
They were there to enjoy us and they were there for us to pray for us.
They were there to enjoy us and they were there for us to pray for us.
They were there to enjoy us and they were there for us to pray for us.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
What is the best way to play Cricket?

Predictions >>>
What is the best way to play cricket?
What is the best way to play cricket?
What is the best way to play cricket?
---------------------------------------------------------



...

---------------------------------------------------------
What are the ways to commit suicide?

Predictions >>>
How can I commit suicide?
How can I commit suicide?
How can I commit suicide?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How can I get started using Quora?

Predictions >>>
How can I get started with Quora?
How can I get started with Quora?
How can I get started with Quora?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How to play chess?

Predictions >>>
How can I play chess?
How can I play chess?
How can I play chess?
---------------------------------------------------------



### Evaluation using Rouge score :-- 
We will now check how our model is performing using Rouge score. 
First we will see accuracy for above sentences.
Below is the accuracy for the model we trained now :--

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=to_predict, references=predicted)
print(results)

{'rouge1': 0.8656432748538013, 'rouge2': 0.7579131652661064, 'rougeL': 0.8654553049289891, 'rougeLsum': 0.8648287385129491}


Accuracy for model trained in Mandate-3 :--

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(encoder_decoder_type="bart", 
                     encoder_decoder_name="/content/gdrive/My Drive/best_model"
                     ,use_cuda=True
                     )

print(type(to_predict))
predicted = []
j = 0
# while True:
for i in to_predict:
    to_predict1 = [i]

    preds = model.predict(to_predict1)

    predicted.insert(j, preds[0])
    print("---------------------------------------------------------")
    print(i)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)
        # predicted[j].append(pred)
    
    j = j+1
    print("---------------------------------------------------------")
    print()

rouge = evaluate.load('rouge')

results = rouge.compute(predictions=to_predict, references=predicted)
print("ACCURACY ::::")
print(results)

<class 'list'>


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
They were there to enjoy us and they were there to pray for us.

Predictions >>>
They were there to enjoy us and they were there also to pray for us.
They were there to enjoy us and they were there also to pray for us.
They were there to enjoy us and they were there also to pray for us.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
What is the best way to play Cricket?

Predictions >>>
What is the best way to play cricket?
How do I play cricket?
How do I play cricket?
---------------------------------------------------------



...

---------------------------------------------------------
What are the ways to commit suicide?

Predictions >>>
How can I commit suicide?
How can I commit suicide?
How can I commit suicide?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How can I get started using Quora?

Predictions >>>
How do I get started with Quora?
How do I get started with Quora?
How do I get started with Quora?
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
How to play chess?

Predictions >>>
How do I play chess?
How do I play chess?
How do I play chess?
---------------------------------------------------------

ACCURACY ::::
{'rouge1': 0.7882248725261429, 'rouge2': 0.6397759103641457, 'rougeL': 0.7779830322933771, 'rougeLsum': 0.7827344942816812}


In [ ]:
!cd data && ls -l

Code for evaluating test data from PAWS dataset :--

In [ ]:
# Google Data
test_df = pd.read_csv("data/test.tsv", sep="\t").astype(str)
test_df = test_df.loc[test_df["label"] == "1"]
test_df = test_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

test_df = test_df[["input_text", "target_text"]]
test_df["prefix"] = "paraphrase"

test_df = test_df[["prefix", "input_text", "target_text"]]

test_df = test_df.dropna()

test_df["input_text"] = test_df["input_text"].apply(clean_unnecessary_spaces)
test_df["target_text"] = test_df["target_text"].apply(clean_unnecessary_spaces)


test_df

,prefix,input_text,target_text
2,paraphrase,"In January 2011, the Deputy Secretary General ...","In January 2011, FIBA Asia deputy secretary ge..."
5,paraphrase,"During her sophomore, junior and senior summer...","During her second, junior and senior summers, ..."
7,paraphrase,"His father emigrated to Missouri in 1868, but ...",His father emigrated to Missouri in 1868 but r...
9,paraphrase,It is situated south of Köroğlu Mountains and ...,It is situated south of Köroğlu - mountains an...
10,paraphrase,The Río Blanco mine is a large copper mine loc...,The Río Blanco - Mine is a large copper mine i...
...,...,...,...
7990,paraphrase,Twice Sparrow sold the island twice to Thomas ...,Sparrow twice sold the island to Thomas Polloc...
7994,paraphrase,The name in Tupi means `` insensitive stone ''...,"The name in Tupi means '' hard stone ``, '' in..."
7995,paraphrase,"The company has branches in Tokyo, based in th...",The company has branches in Tokyo based in Sai...
7997,paraphrase,The modern coat of arms of Bavaria was designe...,The modern coat of arms of Bavaria was designe...


In [ ]:
test_df_accuracy = test_df.head(70)

In [ ]:
test_df_accuracy

,prefix,input_text,target_text
2,paraphrase,"In January 2011, the Deputy Secretary General ...","In January 2011, FIBA Asia deputy secretary ge..."
5,paraphrase,"During her sophomore, junior and senior summer...","During her second, junior and senior summers, ..."
7,paraphrase,"His father emigrated to Missouri in 1868, but ...",His father emigrated to Missouri in 1868 but r...
9,paraphrase,It is situated south of Köroğlu Mountains and ...,It is situated south of Köroğlu - mountains an...
10,paraphrase,The Río Blanco mine is a large copper mine loc...,The Río Blanco - Mine is a large copper mine i...
...,...,...,...
164,paraphrase,The Thrash - Metal - Group Anthrax opened Kiss...,The thrash metal group Anthrax opened for Kiss...
165,paraphrase,"According to the Bureau of Meteorology, the lo...","According to the Bureau of Meteorology, the lo..."
171,paraphrase,He was born in a small kingdom of 1000 `` li '...,He was born in a small kingdom of 1000 `` li '...
173,paraphrase,Holodecks can also be used to encourage studen...,Holodecks can also be used to encourage pupils...


In [ ]:
to_predict_small = [
     input_text
    for  input_text in test_df_accuracy["input_text"].tolist()
]

to_predict_small

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel

model = Seq2SeqModel(encoder_decoder_type="bart", 
                     encoder_decoder_name="/content/gdrive/My Drive/4May/best_model"
                     ,use_cuda=True  
                  # use_cuda=True if you have GPU access else assign it False
                     )

predicted = []
j = 0
# while True:
for i in to_predict_small:
    to_predict1 = [i]

    preds = model.predict(to_predict1)

    predicted.insert(j, preds[0])
    print("---------------------------------------------------------")
    print(i)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)
        # predicted[j].append(pred)
    
    j = j+1
    print("---------------------------------------------------------")
    print()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
In January 2011, the Deputy Secretary General of FIBA Asia, Hagop Khajirian, inspected the venue together with SBP - President Manuel V. Pangilinan.

Predictions >>>
In January 2011 the deputy secretary general of FIBA Asia, Hagop Khajirian, inspected the venue together with SBP President Manuel V. Pangilinan.
In January 2011 the deputy secretary general of FIBA Asia, Hagop Khajirian, inspected the venue together with SBP President Manuel V. Pangilinan.
In January 2011 the deputy secretary general of FIBA Asia, Hagop Khajirian, inspected the venue together with SBP President Manuel V. Pangilinan.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
During her sophomore, junior and senior summers, she spent half of it with her Alaska team, and half playing, and living in Oregon.

Predictions >>>
During her sophomore, junior and senior summers she spent half with her Alaska team and half playing and living in Oregon.
During her sophomore, junior and senior summers she spent half with her Alaska team and half playing and living in Oregon.
During her sophomore, junior and senior summers she spent half of it playing with her Alaska team and half playing in Oregon and living in Portland.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

---------------------------------------------------------
He was born in a small kingdom of 1000 `` li '' northeast of Wa ( Japan ).

Predictions >>>
He was born in a small kingdom of 1000 `` li '' northeast of Wa ( Japan ).
He was born in a small kingdom of 1000 `` li '' northeast of Wa ( Japan ).
He was born in a small kingdom of 1000 `` li '' northeast of Wa ( Japan ).
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
Holodecks can also be used to encourage students to build a scene or to even describe a scene.

Predictions >>>
Holodecks can also be used to encourage students to build a scene or even describe a scene.
Holodecks can also be used to encourage students to build a scene or even describe a scene.
Holodecks can also be used to encourage students to build a scene or even describe a scene.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
Chuck Chuck Campbell of `` Go Knoxville '' compared Ariana Grande in `` Rare '' with Stefani's vocals.

Predictions >>>
Chuck Campbell of `` Go Knoxville '' compared Ariana Grande in `` Rare '' with Stefani's vocals.
Chuck Campbell of `` Go Knoxville '' compared Ariana Grande in `` Rare '' with Stefani's vocals.
Chuck Campbell of `` Go Knoxville '' compared Ariana Grande in `` Rare '' with Stefani's vocals.
---------------------------------------------------------



In [ ]:
# !pip install evaluate
# !pip install rouge-score
import evaluate
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=to_predict_small, references=predicted)
print(results)

{'rouge1': 0.9714710335873413, 'rouge2': 0.9198488512746381, 'rougeL': 0.9534912982368438, 'rougeLsum': 0.953876186023664}


### Checking Semantic Similarity

For checking semantic similarity, we can calculate cosine similarity between the embeddings of the sentences. We use sentence-transformers library to check semantic similarity.
The score shows that two sentences are similar.

In [ ]:
!pip install sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer, util
sentences = ["What is the best way to play cricket?", "How do I play cricket?"]

model = SentenceTransformer('all-MiniLM-L6-v2')

embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

print("Similarity between given sentences is : ", util.pytorch_cos_sim(embedding_1, embedding_2))

Similarity between given sentences is :  tensor([[0.9132]], device='cuda:0')


## Conclusion:

Thus we have implemented Paraphrase model using BART pre-trained model. Below observations were made:
1. After last round of finetuning, model is no more generating questions for an assertive sentence.
2. Rogue score for the model has increased.

Earlier model: rouge1: 0.7882248725261429, rougeL: 0.7779830322933771

Final model: rouge1: 0.8656432748538013, rougeL: 0.8654553049289891

3. Sometimes, the model gives the same sentence with no change in it.

## References:
1. https://huggingface.co/docs/transformers/training
2. https://pub.towardsai.net/fine-tune-bart-for-translation-on-wmt16-dataset-and-train-new-tokenizer-4d0fbdc4aa2e
3. https://simpletransformers.ai/
4. https://www.sbert.net/
5. https://medium.com/@priyankads/rouge-your-nlp-results-b2feba61053a